# Data Preprocessing and Augmentation for Pediatric Pneumonia Detection

## Overview
This notebook handles all data preprocessing tasks including:
- **Image augmentation**: Creating variations of X-ray images to improve model training
- **Data splitting**: Organizing data into training and testing sets
- **Feature extraction**: Extracting statistical features from X-ray images for traditional ML models

## Key Concepts
- **Augmentation**: Artificially increasing dataset size by applying transformations (rotation, flip, etc.)
- **ROI (Region of Interest)**: Dividing images into smaller sections to analyze specific areas
- **Feature extraction**: Converting images into numerical data that traditional ML algorithms can use
- **Denoising**: Removing noise from medical images to improve quality

In [ ]:
# Import required libraries
import cv2  # OpenCV for image processing
import numpy as np  # For numerical operations
import random  # For random transformations
import os  # For file operations
import pandas as pd  # For data handling
import matplotlib.pyplot as plt  # For visualization
from pathlib import Path  # For file path operations
import shutil  # For file copying
from scipy import stats  # For statistical calculations
from tqdm import tqdm  # For progress bars

print("Libraries imported successfully!")

## 1. Image Augmentation Functions

**What is Image Augmentation?**
- Creates new training images by applying small changes to existing ones
- Helps models learn to recognize pneumonia in different conditions
- Prevents overfitting (memorizing specific images instead of learning patterns)

**Techniques Used:**
- **Denoising**: Removes image noise common in X-rays
- **Horizontal flip**: Mirror image left-to-right (lungs are symmetric)
- **Shear**: Slight stretching/skewing of the image
- **Rotation**: Small rotations to simulate different patient positioning

In [ ]:
def augment_image_efficient(image):
    """
    Apply efficient random augmentation techniques to a chest X-ray image.
    
    This function improves image quality and creates variations for training.
    
    Parameters:
    image (numpy.ndarray): Input grayscale X-ray image
    
    Returns:
    numpy.ndarray: Processed and augmented image
    """
    # Ensure image is in the right format
    if len(image.shape) == 3 and image.shape[2] == 1:
        image = image.squeeze()  # Remove extra dimension if present
    h, w = image.shape
    image = image.astype(np.uint8)

    # Step 1: Remove noise from X-ray (denoising)
    # Medical X-rays often have noise that can confuse models
    # h=10: strength of denoising, templateWindowSize=7: comparison window
    denoised_image = cv2.fastNlMeansDenoising(image, None, h=10, templateWindowSize=7, searchWindowSize=21)

    # Step 2: Random horizontal flip (50% chance)
    # Since lungs are symmetric, flipping doesn't change medical meaning
    if random.getrandbits(1):  # Faster than random.random() > 0.5
        cv2.flip(denoised_image, 1, denoised_image)  # Flip horizontally in-place

    # Step 3: Apply random shear (stretching) transformation
    # Simulates slight variations in X-ray positioning
    vertical_shear = np.tan(np.radians(random.uniform(0, 10)))  # 0-10 degrees
    horizontal_shear = np.tan(np.radians(random.uniform(0, 10)))
    
    # Create transformation matrix for shearing
    shear_matrix = np.array([
        [1, horizontal_shear, 0],
        [vertical_shear, 1, 0],
        [0, 0, 1]
    ], dtype=np.float32)
    
    # Apply shear transformation
    denoised_image = cv2.warpPerspective(denoised_image, shear_matrix, (w, h),
                                       flags=cv2.INTER_LINEAR,
                                       borderMode=cv2.BORDER_REPLICATE)

    # Step 4: Random rotation (-3 to +3 degrees)
    # Small rotations simulate different patient positioning
    rotation_angle = random.uniform(-3, 3)
    rotation_matrix = cv2.getRotationMatrix2D((w/2, h/2), rotation_angle, 1.0)
    denoised_image = cv2.warpAffine(denoised_image, rotation_matrix, (w, h),
                                  flags=cv2.INTER_LINEAR,
                                  borderMode=cv2.BORDER_REPLICATE)

    return denoised_image

# Test the augmentation function
print("Image augmentation function defined successfully!")

## 2. Statistical Feature Extraction

**Why Extract Features?**
- Traditional ML algorithms (like SVM) need numerical features, not raw images
- Statistical features capture important image properties
- Can be combined with deep learning for ensemble methods

**Features Extracted:**
- **Basic stats**: Mean, median, min, max (brightness levels)
- **Distribution**: Skewness (asymmetry), kurtosis (tail heaviness)
- **Percentiles**: Values at specific percentage points
- **Energy**: Sum of squared pixel values
- **Entropy**: Measure of randomness/texture

In [ ]:
def extract_features(image):
    """
    Extract statistical features from a grayscale chest X-ray image.
    
    These features capture important statistical properties that can help
    distinguish between normal and pneumonia X-rays.
    
    Parameters:
    image (numpy.ndarray): Grayscale image as a 2D NumPy array
    
    Returns:
    dict: Dictionary containing statistical features
    """
    # Convert image to 1D array for statistical calculations
    pixels = image.flatten().astype(float)
    
    # Initialize dictionary to store features
    features = {}
    
    # Basic statistical measures
    features['maximum'] = np.max(pixels)  # Brightest pixel
    features['minimum'] = np.min(pixels)  # Darkest pixel
    features['mean'] = np.mean(pixels)    # Average brightness
    features['median'] = np.median(pixels) # Middle value when sorted
    features['std_dev'] = np.std(pixels)  # Spread of pixel values
    
    # Mode (most common pixel value)
    features['mode'] = stats.mode(pixels, keepdims=True)[0][0]
    
    # Distribution shape measures
    if np.std(pixels) < 1e-10:  # Handle uniform images
        features['skewness'] = 0.0
        features['kurtosis'] = 0.0
    else:
        # Skewness: asymmetry of distribution (+ = tail on right, - = tail on left)
        skew_val = stats.skew(pixels)
        features['skewness'] = 0.0 if np.isnan(skew_val) else skew_val
        
        # Kurtosis: "tailedness" of distribution (higher = more extreme values)
        kurt_val = stats.kurtosis(pixels)
        features['kurtosis'] = 0.0 if np.isnan(kurt_val) else kurt_val
    
    # Percentiles (values at specific percentage points)
    features['quantile_2.5'] = np.percentile(pixels, 2.5)   # Very dark pixels
    features['quantile_5'] = np.percentile(pixels, 5)       # Dark pixels
    features['quantile_10'] = np.percentile(pixels, 10)     # Darker pixels
    features['quantile_90'] = np.percentile(pixels, 90)     # Brighter pixels
    features['quantile_95'] = np.percentile(pixels, 95)     # Bright pixels
    features['quantile_97.5'] = np.percentile(pixels, 97.5) # Very bright pixels
    
    # Energy measure (sum of squared pixel values)
    features['absolute_energy'] = np.sum(pixels ** 2)
    
    # Entropy (measure of randomness/texture)
    hist, _ = np.histogram(pixels, bins=256, range=(0, 255), density=True)
    hist = hist[hist > 0]  # Remove zeros to avoid log(0)
    features['entropy'] = -np.sum(hist * np.log2(hist))
    
    return features

print("Feature extraction function defined successfully!")

## 3. ROI (Region of Interest) Feature Extraction

**What is ROI Analysis?**
- Divides X-ray image into a grid (e.g., 4x4 = 16 regions)
- Extracts features from each region separately
- Helps identify which parts of lungs show pneumonia signs
- Different grid sizes capture different levels of detail

**Grid Sizes Used:**
- **1x1**: Whole image (global features)
- **4x4**: 16 regions (moderate detail)
- **16x16**: 256 regions (fine detail)

**Benefits:**
- Pneumonia often affects specific lung regions
- Provides spatial information about disease location
- More features can improve traditional ML performance

In [ ]:
def extract_roi_features(image, grid_size=(4, 4)):
    """
    Extract statistical features from each ROI (Region of Interest) in a grid.
    
    This function divides the X-ray into smaller regions and extracts features
    from each region separately, providing spatial information.
    
    Parameters:
    image (numpy.ndarray): Grayscale image as a 2D NumPy array
    grid_size (tuple): Grid dimensions (rows, cols) for ROI division
    
    Returns:
    dict: Dictionary containing statistical features for each ROI
    """
    height, width = image.shape
    rows, cols = grid_size
    
    # Calculate size of each ROI
    roi_height = height // rows
    roi_width = width // cols
    
    # Store all ROI pixel data first (more efficient)
    roi_pixels_list = []
    
    # Extract all ROIs
    for row in range(rows):
        for col in range(cols):
            # Calculate ROI boundaries
            start_row = row * roi_height
            end_row = start_row + roi_height
            start_col = col * roi_width
            end_col = start_col + roi_width
            
            # Extract ROI and flatten to 1D
            roi = image[start_row:end_row, start_col:end_col]
            roi_pixels = roi.flatten().astype(np.float32)
            roi_pixels_list.append(roi_pixels)
    
    # Extract features from each ROI
    all_features = {}
    
    # Percentiles to calculate for efficiency
    percentiles = [2.5, 5, 10, 50, 90, 95, 97.5]  # Include median (50th)
    
    for roi_idx, pixels in enumerate(roi_pixels_list):
        roi_num = roi_idx + 1  # ROI numbering starts from 1
        
        # Basic statistics
        features = {
            f'maximum_roi_{roi_num}': np.max(pixels),
            f'minimum_roi_{roi_num}': np.min(pixels),
            f'mean_roi_{roi_num}': np.mean(pixels),
            f'std_dev_roi_{roi_num}': np.std(pixels)
        }
        
        # Calculate all percentiles at once (more efficient)
        perc_values = np.percentile(pixels, percentiles)
        features[f'quantile_2.5_roi_{roi_num}'] = perc_values[0]
        features[f'quantile_5_roi_{roi_num}'] = perc_values[1]
        features[f'quantile_10_roi_{roi_num}'] = perc_values[2]
        features[f'median_roi_{roi_num}'] = perc_values[3]
        features[f'quantile_90_roi_{roi_num}'] = perc_values[4]
        features[f'quantile_95_roi_{roi_num}'] = perc_values[5]
        features[f'quantile_97.5_roi_{roi_num}'] = perc_values[6]
        
        # Energy calculation
        features[f'absolute_energy_roi_{roi_num}'] = np.sum(pixels * pixels)
        
        # Skip expensive calculations for uniform regions
        std_val = features[f'std_dev_roi_{roi_num}']
        if std_val < 1e-10:  # Nearly uniform region
            features[f'mode_roi_{roi_num}'] = features[f'mean_roi_{roi_num}']
            features[f'skewness_roi_{roi_num}'] = 0.0
            features[f'kurtosis_roi_{roi_num}'] = 0.0
            features[f'entropy_roi_{roi_num}'] = 0.0
        else:
            # Mode calculation
            try:
                mode_result = stats.mode(pixels, keepdims=True)
                features[f'mode_roi_{roi_num}'] = mode_result[0][0]
            except:
                features[f'mode_roi_{roi_num}'] = features[f'mean_roi_{roi_num}']
            
            # Distribution shape
            skew_val = stats.skew(pixels)
            features[f'skewness_roi_{roi_num}'] = 0.0 if np.isnan(skew_val) else skew_val
            
            kurt_val = stats.kurtosis(pixels)
            features[f'kurtosis_roi_{roi_num}'] = 0.0 if np.isnan(kurt_val) else kurt_val
            
            # Entropy (only for reasonably sized ROIs)
            if len(pixels) > 100:
                hist, _ = np.histogram(pixels, bins=128, range=(0, 255), density=True)
                hist = hist[hist > 1e-10]
                if len(hist) > 0:
                    features[f'entropy_roi_{roi_num}'] = -np.sum(hist * np.log2(hist))
                else:
                    features[f'entropy_roi_{roi_num}'] = 0.0
            else:
                features[f'entropy_roi_{roi_num}'] = 0.0
        
        all_features.update(features)
    
    return all_features

print("ROI feature extraction function defined successfully!")

## 4. Data Splitting Utilities

**Why Split Data Properly?**
- Need separate training and testing sets
- Training set: Used to teach the model
- Testing set: Used to evaluate model performance (never seen during training)
- Prevents overfitting by testing on unseen data

**Splitting Strategy:**
- Maintains class balance (equal normal/pneumonia ratios)
- Typically 80% training, 20% testing
- Random shuffle ensures fair distribution

In [ ]:
class DataSplitter:
    """
    A utility class for splitting chest X-ray data into training and testing sets.
    
    This ensures proper data organization for machine learning training while
    maintaining balanced class distributions.
    """
    
    def __init__(self, base_dir=".", train_ratio=0.8):
        """
        Initialize the data splitter.
        
        Args:
            base_dir: Directory containing train and test folders
            train_ratio: Fraction of data for training (0.8 = 80% train, 20% test)
        """
        self.base_dir = Path(base_dir)
        self.train_ratio = train_ratio
        self.test_ratio = 1 - train_ratio
        
        # Define directory paths
        self.original_train_dir = self.base_dir / "train"
        self.original_test_dir = self.base_dir / "test"
        self.new_train_dir = self.base_dir / "new_train"
        self.new_test_dir = self.base_dir / "new_test"
        
        # Medical image classes
        self.classes = ["NORMAL", "PNEUMONIA"]
    
    def aggregate_files(self):
        """Collect all image files from both train and test directories."""
        aggregated = {"NORMAL": [], "PNEUMONIA": []}
        
        for class_name in self.classes:
            # Collect from training directory
            train_class_dir = self.original_train_dir / class_name
            if train_class_dir.exists():
                # Find all common image formats
                files = (list(train_class_dir.glob("*.jpeg")) + 
                        list(train_class_dir.glob("*.jpg")) + 
                        list(train_class_dir.glob("*.png")))
                aggregated[class_name].extend([(f, "train") for f in files])
            
            # Collect from test directory
            test_class_dir = self.original_test_dir / class_name
            if test_class_dir.exists():
                files = (list(test_class_dir.glob("*.jpeg")) + 
                        list(test_class_dir.glob("*.jpg")) + 
                        list(test_class_dir.glob("*.png")))
                aggregated[class_name].extend([(f, "test") for f in files])
        
        return aggregated
    
    def calculate_split_counts(self, pneumonia_count, normal_count):
        """
        Calculate how many files should go to train and test.
        
        Uses pneumonia count as reference to maintain class balance.
        
        Args:
            pneumonia_count: Total number of pneumonia images
            normal_count: Total number of normal images
            
        Returns:
            Dictionary with split counts for each class
        """
        # Calculate target counts based on pneumonia distribution
        target_pneumonia_test = int(pneumonia_count * self.test_ratio)
        target_pneumonia_train = pneumonia_count - target_pneumonia_test
        
        # Match normal test count to pneumonia for balanced testing
        target_normal_test = target_pneumonia_test
        target_normal_train = normal_count - target_normal_test
        
        return {
            "pneumonia": {"train": target_pneumonia_train, "test": target_pneumonia_test},
            "normal": {"train": target_normal_train, "test": target_normal_test}
        }
    
    def create_directories(self):
        """Create new directory structure for split data."""
        # Remove existing directories if they exist
        for dir_path in [self.new_train_dir, self.new_test_dir]:
            if dir_path.exists():
                shutil.rmtree(dir_path)
        
        # Create new directory structure
        for split in ["new_train", "new_test"]:
            for class_name in self.classes:
                dir_path = self.base_dir / split / class_name
                dir_path.mkdir(parents=True, exist_ok=True)
    
    def split_data(self):
        """Main method to split the data according to the specified ratio."""
        print("Aggregating files...")
        aggregated = self.aggregate_files()
        
        pneumonia_files = aggregated["PNEUMONIA"]
        normal_files = aggregated["NORMAL"]
        
        print(f"Total PNEUMONIA files: {len(pneumonia_files)}")
        print(f"Total NORMAL files: {len(normal_files)}")
        
        # Calculate split counts
        split_counts = self.calculate_split_counts(len(pneumonia_files), len(normal_files))
        
        print("\nTarget split counts:")
        print(f"PNEUMONIA - Train: {split_counts['pneumonia']['train']}, Test: {split_counts['pneumonia']['test']}")
        print(f"NORMAL - Train: {split_counts['normal']['train']}, Test: {split_counts['normal']['test']}")
        
        # Create directories and copy files
        self.create_directories()
        
        # Shuffle files for random distribution
        random.shuffle(pneumonia_files)
        random.shuffle(normal_files)
        
        # Split and copy files
        self._copy_split_files(pneumonia_files, normal_files, split_counts)
        
        print("\nData split completed successfully!")
        return split_counts
    
    def _copy_split_files(self, pneumonia_files, normal_files, split_counts):
        """Copy files to their designated train/test directories."""
        print("\nCopying files...")
        
        # Split pneumonia files
        pneumonia_train = pneumonia_files[:split_counts["pneumonia"]["train"]]
        pneumonia_test = pneumonia_files[split_counts["pneumonia"]["train"]:]
        
        # Split normal files
        normal_test = normal_files[:split_counts["normal"]["test"]]
        normal_train = normal_files[split_counts["normal"]["test"]:]
        
        # Copy files to appropriate directories
        for file_path, _ in pneumonia_train:
            dest = self.new_train_dir / "PNEUMONIA" / file_path.name
            shutil.copy2(file_path, dest)
        
        for file_path, _ in pneumonia_test:
            dest = self.new_test_dir / "PNEUMONIA" / file_path.name
            shutil.copy2(file_path, dest)
        
        for file_path, _ in normal_train:
            dest = self.new_train_dir / "NORMAL" / file_path.name
            shutil.copy2(file_path, dest)
        
        for file_path, _ in normal_test:
            dest = self.new_test_dir / "NORMAL" / file_path.name
            shutil.copy2(file_path, dest)

print("Data splitting utility defined successfully!")

## 5. Batch Feature Extraction for Machine Learning

**Purpose:**
- Process entire datasets to create feature files
- Creates CSV files with features for each image
- Used by traditional ML algorithms (SVM, Random Forest)
- Different ROI schemes provide different levels of spatial detail

**Output:**
- CSV files with rows = images, columns = features
- Includes image ID and label (NORMAL=1, PNEUMONIA=0)
- Can be loaded directly into scikit-learn

In [ ]:
def extract_all_features(input_dir, output_file, roi_scheme=(16, 16)):
    """
    Extract features from all images in a dataset and save to CSV.
    
    This function processes entire directories of X-ray images and creates
    feature files suitable for traditional machine learning algorithms.
    
    Parameters:
    input_dir (str): Directory containing NORMAL and PNEUMONIA subdirectories
    output_file (str): Path for output CSV file
    roi_scheme (tuple): Grid size for ROI analysis (rows, cols)
    """
    source_dir = os.path.join(os.getcwd(), input_dir)
    normal_dir = os.path.join(source_dir, 'NORMAL')
    pneumonia_dir = os.path.join(source_dir, 'PNEUMONIA')
    
    data = []
    count = 0
    
    print(f"Extracting features using {roi_scheme[0]}x{roi_scheme[1]} ROI scheme...")
    
    # Process NORMAL images
    print("Processing NORMAL images...")
    if os.path.exists(normal_dir):
        normal_files = os.listdir(normal_dir)
        for filename in tqdm(normal_files, desc="NORMAL"):
            if count % 100 == 0 and count > 0:
                print(f"Processed {count} images...")
            
            path = os.path.join(normal_dir, filename)
            # Load image in grayscale
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            
            # Extract ROI features
            features = extract_roi_features(img, grid_size=roi_scheme)
            features['image_id'] = filename
            features['label'] = 1  # NORMAL = 1
            data.append(features)
            count += 1
    
    # Process PNEUMONIA images
    print("Processing PNEUMONIA images...")
    if os.path.exists(pneumonia_dir):
        pneumonia_files = os.listdir(pneumonia_dir)
        for filename in tqdm(pneumonia_files, desc="PNEUMONIA"):
            if count % 100 == 0 and count > 0:
                print(f"Processed {count} images...")
            
            path = os.path.join(pneumonia_dir, filename)
            # Load image in grayscale
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            
            # Extract ROI features
            features = extract_roi_features(img, grid_size=roi_scheme)
            features['image_id'] = filename
            features['label'] = 0  # PNEUMONIA = 0
            data.append(features)
            count += 1
    
    # Create DataFrame and save to CSV
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)
    
    print(f"\nFeature extraction completed!")
    print(f"Total images processed: {count}")
    print(f"Features per image: {len(df.columns) - 2}")  # Exclude image_id and label
    print(f"Output saved to: {output_file}")
    
    return df

print("Batch feature extraction function defined successfully!")

## 6. Test the Functions

Let's test our preprocessing functions with a sample image to make sure everything works correctly.

In [ ]:
# Test with a sample image from our dataset
sample_dir = "../data/training/augmented_train/NORMAL"
if os.path.exists(sample_dir):
    # Get the first image file
    image_files = [f for f in os.listdir(sample_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    if image_files:
        sample_path = os.path.join(sample_dir, image_files[0])
        
        # Load and test image processing
        img = cv2.imread(sample_path, cv2.IMREAD_GRAYSCALE)
        print(f"Loaded sample image: {sample_path}")
        print(f"Image shape: {img.shape}")
        
        # Test augmentation
        augmented_img = augment_image_efficient(img)
        print(f"Augmentation successful! Output shape: {augmented_img.shape}")
        
        # Test feature extraction
        features = extract_features(img)
        print(f"\nExtracted {len(features)} global features")
        print("Sample features:", list(features.keys())[:5])
        
        # Test ROI feature extraction
        roi_features = extract_roi_features(img, grid_size=(4, 4))
        print(f"\nExtracted {len(roi_features)} ROI features (4x4 grid)")
        print("Sample ROI features:", list(roi_features.keys())[:5])
        
        # Visualize original vs augmented
        plt.figure(figsize=(12, 4))
        
        plt.subplot(1, 3, 1)
        plt.imshow(img, cmap='gray')
        plt.title('Original Image')
        plt.axis('off')
        
        plt.subplot(1, 3, 2)
        plt.imshow(augmented_img, cmap='gray')
        plt.title('Augmented Image')
        plt.axis('off')
        
        plt.subplot(1, 3, 3)
        plt.imshow(np.abs(img.astype(float) - augmented_img.astype(float)), cmap='hot')
        plt.title('Difference (Hot = More Change)')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        print("\nAll preprocessing functions working correctly!")
    else:
        print("No image files found in the sample directory")
else:
    print(f"Sample directory not found: {sample_dir}")
    print("Make sure the dataset is properly copied to ../data/training/")

## 7. Summary

This notebook provides all the data preprocessing tools needed for the pneumonia detection project:

### Completed Functions:
1. **Image Augmentation**: Improves X-ray quality and creates training variations
2. **Feature Extraction**: Converts images to numerical features for traditional ML
3. **ROI Analysis**: Spatial feature extraction from image regions
4. **Data Splitting**: Proper train/test separation with class balance
5. **Batch Processing**: Automated feature extraction for entire datasets

### Next Steps:
- Use these functions in model training notebooks
- Create feature CSV files for SVM and ensemble methods
- Apply augmentation during deep learning training

### Key Outputs:
- Augmented images for training
- Feature CSV files for traditional ML
- Properly split train/test datasets

**All preprocessing tools are now ready for use in subsequent notebooks!**